In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from skimage import io
print(os.listdir('Dataset'))

['Direction_left', 'Direction_right', 'Fist', 'Five-palm', 'Four', 'OK', 'One', 'Stop', 'Three', 'Thumbs_down', 'Thumbs_up', 'Two']


In [3]:
test_ratio = 0.2
validation_ratio = 0.2
batch_size = 50
epochs = 10
dataset_dir = 'Dataset'
all_images = []
class_ids = []
import os
n_classes = len(os.listdir(dataset_dir))
print(n_classes)

alldirs = []
for d in os.listdir(dataset_dir):
    alldirs.append(f"{dataset_dir}/{d}")
print(alldirs)

12
['Dataset/Direction_left', 'Dataset/Direction_right', 'Dataset/Fist', 'Dataset/Five-palm', 'Dataset/Four', 'Dataset/OK', 'Dataset/One', 'Dataset/Stop', 'Dataset/Three', 'Dataset/Thumbs_down', 'Dataset/Thumbs_up', 'Dataset/Two']


In [33]:
import re
import cv2
import numpy as np
for d in alldirs:
    for img in os.listdir(d):
        label = re.findall('[l]=[0-9]+', img)
        label = label[0][2:]
        
        current_img = f"{d}/{img}"
        image = cv2.imread(current_img)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        all_images.append(gray_image.reshape(89, 100, 1))
        class_ids.append(label)
    print(f"{d} done.")
            
all_images = np.array(all_images)
class_ids = np.array(class_ids)


Dataset/Direction_left done.
Dataset/Direction_right done.
Dataset/Fist done.
Dataset/Five-palm done.
Dataset/Four done.
Dataset/OK done.
Dataset/One done.
Dataset/Stop done.
Dataset/Three done.
Dataset/Thumbs_down done.
Dataset/Thumbs_up done.
Dataset/Two done.


In [35]:
outputSize = len(os.listdir(dataset_dir)) 
epochs = 30 # Number of epochs

In [36]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [37]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [40]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(all_images, class_ids, test_size=test_ratio)

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=validation_ratio)


# Doing the same as above for the validation directory
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)
y_validation = to_categorical(y_validation, n_classes)



In [49]:

test_datagen = ImageDataGenerator(rescale=1./255)
# Setting up the train generator to flow from the train directory
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size = (256,256),
    batch_size = 32
   
)

# Doing the same as above for the validation directory
val_generator = test_datagen.flow_from_directory(
    dataset_dir,
    target_size = (256,256),
    batch_size = 32
    
)


Found 12000 images belonging to 12 classes.
Found 12000 images belonging to 12 classes.


In [41]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
def create_model(outputSize):
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (256,256,1)))
    model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Flatten())
    model.add(Dropout(rate = 0.5))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(units = outputSize, activation = 'softmax'))
    model.compile(optimizer = Adam(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model
# Creating the model

In [42]:
model = create_model(outputSize)

In [21]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 conv2d_7 (Conv2D)           (None, 252, 252, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 126, 126, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 124, 124, 64)      18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 122, 122, 64)      36928     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 61, 61, 64)        0         
 g2D)                                                 

In [24]:
import tensorflow as tf
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [25]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=outputSize*1000/32,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=outputSize*500/32
)

C:\Users\devar\AppData\Local\Temp\ipykernel_6172\3681782265.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
375/375 [==============================] - 1951s 5s/step - loss: 1.2923 - accuracy: 0.5135 - val_loss: 0.4573 - val_accuracy: 0.8152
Epoch 2/30
148/375 [==========>...................] - ETA: 17:50 - loss: 0.6607 - accuracy: 0.7361

KeyboardInterrupt: 